In [ ]:
import os,glob,sys
from tqdm.notebook import tqdm
import datetime

import numpy as np

import pandas as pd

import scipy as sc
import scipy.signal as signal

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec


In [ ]:
outFolder = 'duneParam'

# read the data from example_data, convert strings in DatesFrame to datetime values
ZFrame = pd.read_csv('Z_locations.csv',index_col=0,skiprows = 6)
MXYFrame = pd.read_csv('MXY_locations.csv',index_col=0,skiprows = 6)
dateTable_10yr = pd.read_csv('dates.csv',index_col=0,skiprows = 6)
for i,_ in enumerate(dateTable_10yr.columns):
    dateTable_10yr.iloc[:,i] = pd.to_datetime(dateTable_10yr.iloc[:,i])

# make the values of M and Z a numpy arrays
Z_10yr = ZFrame.values
M_10yr = MXYFrame.M.values

Z_10yrf = signal.savgol_filter(Z_10yr,41,3,mode='mirror',axis = 0)

# 10 years
CrestTroughs_10yr = pd.read_pickle(os.path.join(outFolder,'CrestTroughLocations_2011-2021_centerline.pkl'))

DuneParam_10yr = pd.read_pickle(os.path.join(outFolder,f'DuneParam_all_2011-2021_centerline.pkl'))
DuneParamStat_10yr = pd.read_pickle(os.path.join(outFolder,f'DuneParamStat_all_2011-2021_centerline.pkl'))

duneSpeeds_10yr = pd.read_pickle(os.path.join(outFolder,f'DuneSpeeds_all_2011-2021_centerline.pkl'))
duneSpeedCorr_10yr = pd.read_pickle(os.path.join(outFolder,f'duneSpeedCorr_all_2011-2021_centerline.pkl'))
duneDisplacement_10yr = pd.read_pickle(os.path.join(outFolder,f'duneDisplacement_all_2011-2021_centerline.pkl'))

# discharges at Tiel, 10 years daily average
afvoeren = pd.read_csv('QTiel_2009_2020_daymean.csv', index_col=0, parse_dates = True)


In [ ]:
dates_10yr = dateTable_10yr.loc[1000:5000].mode().T# dates 2018 have are number 156:214
dates_10yr.columns = ['date']
dates_10yr.iloc[30].date = datetime.datetime.strptime('2012-05-31', '%Y-%m-%d')
dates_10yr['date_5']  = dates_10yr.date - datetime.timedelta(days =5)

## FIGUREN PAPER 1
Submitted to Geophysical Research Letters: 2021-11-23


In [ ]:
## LINKER KOLOM FIGUUR!
fig, ax = plt.subplots(4,2,figsize=(7.3,8))

QTiel5D = []
for index, row in dates_10yr.iterrows():
    QTiel5D = np.append(QTiel5D,afvoeren.loc[row.date_5:row.date].mean())
dates = dates_10yr.date.values
    
ax[0,0].scatter(QTiel5D[:],
                DuneParamStat_10yr.H.values[:], marker = '.',s=30,
                color = 'xkcd:orange', alpha = 0.5, label= 'Dune height')
ax[0,0].set_ylabel('$\delta$ [m]')
ax[0,0].grid(alpha = 0.75, zorder = 10)
ax[0,0].set_ylim(0.4,1.2)

ax[1,0].scatter(QTiel5D[:],
                DuneParamStat_10yr.L.values[:], marker = '.',s=30,
                color = 'xkcd:blue', alpha = 0.5, label= 'Dune length')
ax[1,0].set_ylabel('$\lambda$ [m]')
ax[1,0].grid(alpha = 0.5, zorder = 10)
ax[1,0].set_ylim(60,150)

ax[2,0].scatter(QTiel5D[:],
                DuneParamStat_10yr.Slee.values[:], marker = '.',s=30,
                color = 'xkcd:green', alpha = 0.5, label= 'Lee slope angle')
ax[2,0].set_ylabel('$\\alpha$ [deg]')
ax[2,0].grid(alpha = 0.5, zorder = 10)
ax[2,0].set_ylim(0.75,3.25)

ax[3,0].scatter(QTiel5D[:],
                DuneParamStat_10yr.LH.values[:], marker = '.',s=30,
                color = 'xkcd:grey', alpha = 0.5, label= 'Middle Waal section')
ax[3,0].set_xlabel('Q$_{5days}$ [m$^3$/s]')
ax[3,0].set_ylabel('$\psi$ [-]')
ax[3,0].grid(alpha = 0.5, zorder = 10)
ax[3,0].set_ylim(0.002,0.018)

ax[1,0].get_shared_x_axes().join(ax[0,1], ax[1,1])         
ax[2,0].get_shared_x_axes().join(ax[0,1], ax[2,1])           
ax[3,0].get_shared_x_axes().join(ax[0,1], ax[3,1])

# ax[0,0].set_xlim(500,4500)
plt.setp([a.get_xticklabels() for a in ax[:-1,0]], visible=False)

## RECHTER KOLOM FIGUUR
l = 75

axQ = ax[0,1].twinx()
axQ.plot(afvoeren.index,afvoeren.Q, color = 'black', alpha = 0.25)
axQ.set_ylabel('$Q$ [$m^3/s$]', color = 'xkcd:dark grey')
axQ.tick_params(axis='y', colors='xkcd:dark grey')
axQ.spines['right'].set_color('xkcd:dark grey')

ax[0,1].fill_between(dates[156:196],
                     DuneParamStat_10yr.H05.values[156:196].astype(float),
                     DuneParamStat_10yr.H95.values[156:196].astype(float),
                     color = 'xkcd:orange', alpha = 0.2, label = '90% interval')
ax[0,1].fill_between(dates[156:196],
                     DuneParamStat_10yr.H25.values[156:196].astype(float),
                     DuneParamStat_10yr.H75.values[156:196].astype(float),
                     color = 'xkcd:orange', alpha = 0.4,label =  '50% interval')
ax[0,1].scatter(dates[156:196],
                DuneParamStat_10yr.H.values[156:196],marker = '.',s = 30,
                color = 'xkcd:dark orange', label= 'Dune height')
ax[0,0].scatter(QTiel5D[156:196],
                DuneParamStat_10yr.H.values[156:196],marker = '.',s = 40,
                color = 'xkcd:dark orange', label= 'Dune height')
ax[0,1].set_ylabel('$\delta$ [m]')
ax[0,1].set_ylim(0,2.5)


axQ = ax[1,1].twinx()
axQ.plot(afvoeren.index,afvoeren.Q, color = 'black', alpha = 0.25)
axQ.set_ylabel('$Q$ [$m^3/s$]', color = 'xkcd:dark grey')
axQ.tick_params(axis='y', colors='xkcd:dark grey')
axQ.spines['right'].set_color('xkcd:dark grey')

ax[1,1].fill_between(dates[156:196],
                     DuneParamStat_10yr.L05.values[156:196].astype(float),
                     DuneParamStat_10yr.L95.values[156:196].astype(float),
                     color = 'xkcd:blue', alpha = 0.2, label = '90% interval')
ax[1,1].fill_between(dates[156:196],
                     DuneParamStat_10yr.L25.values[156:196].astype(float),
                     DuneParamStat_10yr.L75.values[156:196].astype(float),
                     color = 'xkcd:blue', alpha = 0.4, label = '50% interval')
ax[1,1].scatter(dates[156:196],
                DuneParamStat_10yr.L.values[156:196],marker = '.',s = 30,
                color = 'xkcd:dark blue', label= 'Dune length')
ax[1,0].scatter(QTiel5D[156:196],
                DuneParamStat_10yr.L.values[156:196],marker = '.',s = 40,
                color = 'xkcd:dark blue', label= 'Dune length')
ax[1,1].set_ylabel('$\lambda$ [m]')
ax[1,1].set_ylim(0,300)


axQ = ax[2,1].twinx()
axQ.plot(afvoeren.index,afvoeren.Q, color = 'black', alpha = 0.25)
axQ.set_ylabel('$Q$ [$m^3/s$]', color = 'xkcd:dark grey')
axQ.tick_params(axis='y', colors='xkcd:dark grey')
axQ.spines['right'].set_color('xkcd:dark grey')

ax[2,1].fill_between(dates[156:196],
                     DuneParamStat_10yr.Slee05.values[156:196].astype(float),
                     DuneParamStat_10yr.Slee95.values[156:196].astype(float),
                     color = 'xkcd:green', alpha = 0.2, label = '90% interval')
ax[2,1].fill_between(dates[156:196],
                     DuneParamStat_10yr.Slee25.values[156:196].astype(float),
                     DuneParamStat_10yr.Slee75.values[156:196].astype(float),
                     color = 'xkcd:green', alpha = 0.4, label = '50% interval')
ax[2,1].scatter(dates[156:196],
                DuneParamStat_10yr.Slee.values[156:196],marker = '.',s = 30,
                color = 'xkcd:dark green', label= 'Lee slope angle')
ax[2,0].scatter(QTiel5D[156:196],
                DuneParamStat_10yr.Slee.values[156:196],marker = '.',s = 40,
                color = 'xkcd:dark green', label= 'Lee slope angle')
ax[2,1].set_ylabel('$\\alpha$ [deg]')
ax[2,1].set_ylim(0,6)

axQ = ax[3,1].twinx()
axQ.plot(afvoeren.index,afvoeren.Q, color = 'black', alpha = 0.25)
axQ.set_ylabel('$Q$ [$m^3/s$]', color = 'xkcd:dark grey')
axQ.tick_params(axis='y', colors='xkcd:dark grey')
axQ.spines['right'].set_color('xkcd:dark grey')

ax[3,1].fill_between(dates[156:196],
                     DuneParamStat_10yr.LH05.values[156:196].astype(float),
                     DuneParamStat_10yr.LH95.values[156:196].astype(float),
                     color = 'xkcd:grey', alpha = 0.2, label = '90% interval')
ax[3,1].fill_between(dates[156:196],
                     DuneParamStat_10yr.LH25.values[156:196].astype(float),
                     DuneParamStat_10yr.LH75.values[156:196].astype(float),
                     color = 'xkcd:grey', alpha = 0.4, label = '50% interval')
ax[3,1].scatter(dates[156:196],
                DuneParamStat_10yr.LH.values[156:196],marker = '.',s = 30,
                color = 'xkcd:dark grey')#, label= 'Aspect ratio')
ax[3,0].scatter(QTiel5D[156:196],
                DuneParamStat_10yr.LH.values[156:196],marker = '.',s = 40,
                color = 'xkcd:dark grey')#, label= 'Aspect ratio')
ax[3,1].set_ylabel('$\psi$ [-]')
# ax[3,1].set_xlim(dates[0],dates[40])
ax[3,1].set_ylim(0,0.04)
for tick in ax[3,1].get_xticklabels():
    tick.set_rotation(45)

ax[3,1].legend()
         
ax[1,1].get_shared_x_axes().join(ax[0,1], ax[1,1])         
ax[2,1].get_shared_x_axes().join(ax[0,1], ax[2,1])           
ax[3,1].get_shared_x_axes().join(ax[0,1], ax[3,1])

ax[0,1].set_xlim(dates[156],dates[196])
plt.setp([a.get_xticklabels() for a in ax[:-1,1]], visible=False)


ax[0,1].text(0.05, 0.90, "b",fontsize = 14,
             horizontalalignment='center',
             verticalalignment='center',
             transform = ax[0,1].transAxes)
ax[1,1].text(0.05, 0.90, "d",fontsize = 14,
             horizontalalignment='center',
             verticalalignment='center',
             transform = ax[1,1].transAxes)
ax[2,1].text(0.05, 0.90, "f",fontsize = 14,
             horizontalalignment='center',
             verticalalignment='center',
             transform = ax[2,1].transAxes)
ax[3,1].text(0.05, 0.90, "h",fontsize = 14,
             horizontalalignment='center',
             verticalalignment='center',
             transform = ax[3,1].transAxes)


ax[0,0].text(0.05, 0.90, "a",fontsize = 14,
             horizontalalignment='center',
             verticalalignment='center',
             transform = ax[0,0].transAxes)
ax[1,0].text(0.05, 0.90, "c",fontsize = 14,
             horizontalalignment='center',
             verticalalignment='center',
             transform = ax[1,0].transAxes)
ax[2,0].text(0.05, 0.90, "e",fontsize = 14,
             horizontalalignment='center',
             verticalalignment='center',
             transform = ax[2,0].transAxes)
ax[3,0].text(0.05, 0.90, "g",fontsize = 14,
             horizontalalignment='center',
             verticalalignment='center',
             transform = ax[3,0].transAxes)
                  
fig.tight_layout()
# fig.savefig('GRL_DuneParam.png',dpi = 300)



In [ ]:
ist = 156
iend = ist+40

crests = CrestTroughs_10yr.Crests.loc[ist:iend]
dates = dateTable_10yr.iloc[1000:2000].mode().values[0]
dates[30] = datetime.datetime.strptime('2012-05-31', '%Y-%m-%d')

ds = duneSpeeds_10yr
corr = duneSpeedCorr_10yr

ds = np.array(ds.values[:,1:], dtype = 'float')
corr = np.array(corr.values[:,1:], dtype = 'float')

ds[np.where(np.abs(ds)<0.05)] = np.nan
ds[:,124] = np.nan

dt = np.diff(dates).astype('timedelta64[D]').astype('int')
dtsum = np.append(0,np.cumsum(dt))

p5,p25,p50,p75,p95 = np.nanpercentile(ds,[5,25,50,75,95], axis = 0)
p5 = np.append(np.nan, p5)
p25 = np.append(np.nan, p25)
p50 = np.append(np.nan, p50)
p75 = np.append(np.nan, p75)
p95 = np.append(np.nan, p95)

Qmean = []
Qmax = []
Qmin = []
# dates = dateTable.iloc[1000:2000].mode().values[0]
for i, date in enumerate(dates):
    if i == len(dates)-1:
        continue
    q = afvoeren.loc[dates[i]:dates[i+1]]
    Qmean = np.append(Qmean,q.mean())
    Qmax = np.append(Qmax,q.max())
    Qmin = np.append(Qmin,q.min())

dates = dates[ist:iend]    
    
# create the figure

fig =  plt.figure(figsize=(7.3,7.3))#constrained_layout=True, 
gs = fig.add_gridspec(10,10)
gs.update(wspace=0.05, hspace = 10)

ax_Q = fig.add_subplot(gs[:,0:1])
# afvoeren.loc[dates[0]:dates[-1]].plot(ax = ax_Q, color = 'xkcd:black', alpha = 0.75)
ax_Q.plot(afvoeren.loc[dates[0]:dates[-1]].Q,
          afvoeren.loc[dates[0]:dates[-1]].index, 
          color = 'black', alpha = 0.75)
for date in dates:
    ax_Q.axhline(date, color = 'xkcd:grey', alpha =0.5)
ax_Q.set_xlim(500,2300)

for tick in ax_Q.get_xticklabels():
    tick.set_rotation(45)

ax_h = fig.add_subplot(gs[:,1:5])
for i,d in enumerate(dtsum[ist:iend]):
    d = d-dtsum[ist]
    ax_h.plot(M_10yr[:],
              Z_10yrf[:,i+ist]*7+d,
              color = 'xkcd:black', alpha = 0.5)
    ax_h.scatter(M_10yr[crests.iloc[i][0]],
                 Z_10yrf[crests.iloc[i][0],i+ist]*8+d,
                 color = 'xkcd:orange', marker = '.',zorder = 10)
ax_h.plot([2100,2485],[240,400], color = 'xkcd:green',zorder = 11, alpha = 0.75, linewidth = 2)
ax_h.plot([1600,2100],[140,240], color = 'xkcd:green',zorder = 11, alpha = 0.75, linewidth = 2)
ax_h.set_xlim(1001,3000)
ax_h.yaxis.set_ticklabels([])

ax_Q.set_ylim(dates[10], dates[29])
ax_h.set_ylim(dtsum[ist+10]-dtsum[ist], dtsum[ist+29]-dtsum[ist])

ax_Q.set_xlim(300,2500)
ax_Q.set_xlabel('Q [m$^3$/s]')
ax_h.set_xlabel('x along axis [m]')


ax_cQ = fig.add_subplot(gs[:5,6:])

# ax_cQ.scatter(Qmean[:],p50[1:],marker = '.', s = 50,
#               color = 'xkcd:light blue',zorder =-1, alpha = 0.5)
ax_cQ.errorbar(Qmean[ist:iend-1],p50[ist+1:iend],
               yerr=[p50[ist+1:iend]-p25[ist+1:iend],p75[ist+1:iend]-p50[ist+1:iend]],
               xerr=[Qmean[ist:iend-1]-Qmin[ist:iend-1],Qmax[ist:iend-1]-Qmean[ist:iend-1]],
               linestyle = '', color = 'xkcd:blue', alpha = 0.5,zorder = -1)
ax_cQ.scatter(Qmean[ist:iend-1],p50[ist+1:iend],marker = '.', s = 50,
              color = 'xkcd:dark blue',zorder =10) # add whiskers 90% of c and min max Q over interval

ax_cQ.set_xlabel('Q [m$^3$/s]')
ax_cQ.set_ylabel('$c$  [m/day]')
ax_cQ.set_ylim(0,7)


ax_cD = fig.add_subplot(gs[5:,6:])

axD = ax_cD.twinx()
axD.plot(afvoeren.loc[dates[0]:dates[-1]].index,
         afvoeren.loc[dates[0]:dates[-1]].Q,
         color = 'black', alpha = 0.25)
axD.set_ylabel('Q [$m^3/s$]', color = 'xkcd:dark grey')
axD.tick_params(axis='y', colors='xkcd:dark grey')
axD.spines['right'].set_color('xkcd:dark grey')

ax_cD.fill_between(dates,
              p25[ist:iend],
              p75[ist:iend],
              color = 'xkcd:orange',alpha = 0.4, label = '50% interval')

ax_cD.fill_between(dates,
              p5[ist:iend],
              p95[ist:iend],
              color = 'xkcd:orange',alpha = 0.2, label = '90% interval')
ax_cD.scatter(dates,
              p50[ist:iend],marker = '.', s = 35,
              color = 'xkcd:dark blue',
              alpha = 0.75, label = 'celerity')

for tick in ax_cD.get_xticklabels():
    tick.set_rotation(45)
    
# ax_cD.set_xlim(datetime.datetime.strptime('2017-11-10',"%Y-%m-%d"),
#                datetime.datetime.strptime('2019-06-01',"%Y-%m-%d"))
ax_cD.set_xlim(dates[0],
               dates[-1])
ax_cD.set_ylabel('$c$  [m/day]')
ax_cD.set_ylim(0,10)
ax_cD.legend()

ax_Q.text(600,datetime.datetime.strptime('2018-12-13',"%Y-%m-%d"), "a", fontsize = 14)
ax_h.text(1100,405, "b", fontsize = 14)
ax_cQ.text(600,5.8,"c", fontsize = 14)
ax_cD.text(datetime.datetime.strptime('2017-12-1',"%Y-%m-%d"),8.8,"d", fontsize = 14)

# fig.tight_layout()
# fig.savefig('GRL_Celerity.png', dpi =300)